In [ ]:
import torch
import torchvision as tv
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook as tqdm

In [ ]:
# Load Alexnet image classifier.
model = tv.models.alexnet(pretrained=True)

In [ ]:
# Show the architecture of the classifier.
print(model)

In [ ]:
# Load a lion image. From this image we want to create an
# adversarial examples.
img_lion = Image.open("lion.jpg")
plt.imshow(img_lion)
plt.show()

In [ ]:
# Define some convenient functions.

as_tensor = tv.transforms.ToTensor()

normalize = tv.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], 
    std=[0.229, 0.224, 0.225]
)

# Reverse the normalization and convert the tensor into a PIL image.
reverse = tv.transforms.Compose([
    tv.transforms.Normalize(
            mean=[0, 0, 0], std=[1.0/0.229, 1.0/0.224, 1.0/0.225]),
    tv.transforms.Normalize(
            mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    tv.transforms.ToPILImage()
])

# Returns the class name for the given index.
def classname(idx):
    import json
    classidx = json.load(open("imagenet_class_index.json"))
    return classidx[str(idx)][1]

# Use Alexnet to predict the category of the given image.
def predict(img):
    # Convert the image to a tensor and normalize it.
    v = normalize(as_tensor(img.copy()))
    # Insert a dimension.
    v = v.unsqueeze(0)
    # Compute class probabilities for the normalized input using Alexnet.
    r = F.softmax(model(v), dim=1)
    # Select the category with the highest probability.
    idx = r.argmax().item()
    # Get the class name for the category.
    label = classname(idx)
    return idx, label, r.data[0, idx].item()

In [ ]:
# Predict the lion image.
predict(img_lion)

In [ ]:
# Load an image of an ostrich.
tmp = Image.open("ostrich.jpg")
plt.imshow(tmp)
plt.show()

In [ ]:
# Predict the category of the ostrich to get the correct
# target category.
predict(tmp)

In [ ]:
# Convert the lion image into a tensor.
img = normalize(as_tensor(img_lion.copy())).requires_grad_(True)

# Set the target category to the category of an ostrich.
target = torch.LongTensor([9])

# Use Adam as the optimizer.
opt = torch.optim.Adam([img])

h = []
for _ in tqdm(range(100)):
    # Bound the entries of the tensor between [-1.8, 1.8]
    x = img.clamp(-1.8, 1.8)
    # Set the correct dimensions so that we can classify x.
    x = x.view(1, 3, 224, 224)
    # Compute the error for x.
    loss = F.cross_entropy(model(x), target)
    h.append(loss.item())
    # Compute the gradient and update the parameters.
    opt.zero_grad()
    loss.backward()
    opt.step()

In [ ]:
# Plot the learning curve.
plt.plot(h)
plt.show()

In [ ]:
img = img.clamp(-1.8, 1.8)

# Convert the tensor (adversarial image) into an image.
img_lion_ostrich = reverse(img.clone())

# Show the adversarial image and the predicted category.
plt.imshow(img_lion_ostrich)
plt.show()
print(predict(img_lion_ostrich))

# Show the original lion image and the predicted category.
plt.imshow(img_lion)
plt.show()
print(predict(img_lion))

In [ ]:
# Show the difference (boosted by a factor of 50).
d = np.abs(
    np.array(img_lion, dtype=np.int) - 
    np.array(img_lion_ostrich, dtype=np.int)
)
d = np.clip(d*50, 0, 255)
plt.imshow(d)
plt.show()